In [102]:
import requests
import os
%pip install pytesseract
%pip install openpyxl

# === Config ===
FEATURE_LAYER_URL = "https://services5.arcgis.com/s1DFLqWfkSIcXZ9O/ArcGIS/rest/services/survey123_24aa1572e50e42e8aa193dad894f57af_results/FeatureServer/0"
TOKEN = "mzFcMRqhxzPAoRJavp2MJg-w3Fau_D0Ix0Qviif4Gs2B6QMLbfdRhemy-rS5IGGgZCF2xD3K_WcpdZ8nn-17hd3ZFfIcZAnk5Rce0_Vih8PwNhBJJy-RhxV1Ni6BIc6dv6sxa-nKL3nwTZZMVHRTukzlRFVOEv5Hci8x9SItu_eAloH56MJ-pE_qlZpnfLki"  # Securely retrieve from .env or secret manager

# === Get last OBJECTID ===
def get_latest_object_id():
    query_url = f"{FEATURE_LAYER_URL}/query"
    params = {
        "where": "1=1",
        "orderByFields": "OBJECTID desc",
        "returnIdsOnly": "true",
        "f": "json",
        "token": TOKEN
    }
    response = requests.get(query_url, params=params)
    if response.status_code == 200:
        ids = response.json().get("objectIds", [])
        return ids[0] if ids else None
    else:
        raise Exception("Failed to get OBJECTID list")

# === Get attachment metadata ===
def get_attachment_info(object_id):
    url = f"{FEATURE_LAYER_URL}/{object_id}/attachments"
    params = {"f": "json", "token": TOKEN}
    response = requests.get(url, params=params)
    return response.json().get("attachmentInfos", []) if response.status_code == 200 else []

# === Download the image file ===
def download_image(attachment, object_id):
    att_id = attachment['id']
    filename = attachment['name'].replace(" ", "_")
    image_url = f"{FEATURE_LAYER_URL}/{object_id}/attachments/{att_id}?token={TOKEN}"
    local_path = os.path.join("downloaded_images", f"{object_id}_{filename}")

    os.makedirs("downloaded_images", exist_ok=True)
    with open(local_path, "wb") as f:
        response = requests.get(image_url)
        f.write(response.content)
    return local_path


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: openpyxl in c:\users\pc\appdata\local\programs\python\python311\lib\site-packages (3.1.5)




[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [103]:
def predict_material(image_path):
    API_URL = "https://pipe-ai-model.onrender.com/predict"
    with open(image_path, "rb") as f:
        files = {"file": f}
        res = requests.post(API_URL, files=files)
    
    if res.status_code == 200:
        return res.json()
    else:
        raise Exception(f"Prediction failed: {res.status_code} - {res.text}")


In [104]:
import csv
from datetime import datetime

def save_result(result, image_path):
    output_path = "predictions_log.csv"
    header = ["timestamp", "filename", "prediction"]
    row = [datetime.now().isoformat(), os.path.basename(image_path), result["prediction"]]

    file_exists = os.path.exists(output_path)
    with open(output_path, "a", newline="") as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(header)
        writer.writerow(row)

    print(f"✅ Prediction saved: {row}")


In [105]:
def run_pipeline():
    try:
        print("🔍 Checking for new Survey123 submission...")
        oid = get_latest_object_id()
        if not oid:
            print("No submissions found.")
            return

        attachments = get_attachment_info(oid)
        if not attachments:
            print(f"No attachments found for OBJECTID {oid}")
            return

        print(f"📥 Downloading image from OBJECTID {oid}")
        image_path = download_image(attachments[0], oid)

        print(f"🧠 Sending image to AI model...")
        prediction = predict_material(image_path)

        print(f"📊 Prediction: {prediction['prediction']}")
        save_result(prediction, image_path)

    except Exception as e:
        print(f"❌ Pipeline failed: {e}")

# Run
if __name__ == "__main__":
    run_pipeline()


🔍 Checking for new Survey123 submission...
📥 Downloading image from OBJECTID 2
🧠 Sending image to AI model...
📊 Prediction: PVC
✅ Prediction saved: ['2025-04-02T20:44:44.781844', '2_1_Inch_Inside_SCH_40_PVC_Pipe.jpg', 'PVC']


In [106]:
  # ✅ LEAD SERVICE LINE IDENTIFIER (PIPE + TAP CARD + ADDRESS)



import requests
import os
import pytesseract
from PIL import Image
import imagehash
from io import BytesIO
import pandas as pd

# === CONFIG ===
FEATURE_LAYER_URL = "https://services5.arcgis.com/s1DFLqWfkSIcXZ9O/ArcGIS/rest/services/survey123_24aa1572e50e42e8aa193dad894f57af_results/FeatureServer/0"
TOKEN = "mzFcMRqhxzPAoRJavp2MJg-w3Fau_D0Ix0Qviif4Gs2B6QMLbfdRhemy-rS5IGGgZCF2xD3K_WcpdZ8nn-17hd3ZFfIcZAnk5Rce0_Vih8PwNhBJJy-RhxV1Ni6BIc6dv6sxa-nKL3nwTZZMVHRTukzlRFVOEv5Hci8x9SItu_eAloH56MJ-pE_qlZpnfLki"
AI_MODEL_URL = "https://pipe-ai-model.onrender.com/predict"
KNOWN_ADDRESSES_FILE = "ocr_data/StudentPotHoleSheet_FletcherDr.xlsx"


# === STEP 1: GET LATEST RECORD ===
# def get_latest_object_id():
#     url = f"{FEATURE_LAYER_URL}/query"
#     params = {
#         "where": "1=1",
#         "outFields": "*",
#         "orderByFields": "OBJECTID DESC",
#         "resultRecordCount": 1,
#         "f": "json",
#         "token": TOKEN
#     }
#     r = requests.get(url, params=params)
#     data = r.json()
#     print(data['features'][0]['attributes'])
#     return data['features'][0]['attributes'], data['features'][0]['attributes']['OBJECTID']

def get_latest_object_id():
    url = f"{FEATURE_LAYER_URL}/query"
    params = {
        "where": "1=1",
        "outFields": "*",
        "orderByFields": "CreationDate DESC",
        "resultRecordCount": 1,
        "f": "json",
        "token": TOKEN
    }
    r = requests.get(url, params=params)
    data = r.json()
    
    if data.get("features"):
        latest = data['features'][0]
        record = latest.get("attributes", latest)  # fallback in case not nested
        return record, record['objectid']
    else:
        raise ValueError("No records found in the survey data.")


In [107]:
# === STEP 2: DOWNLOAD ATTACHMENT FOR LATEST ===
def download_latest_attachment(object_id):
    att_url = f"{FEATURE_LAYER_URL}/{object_id}/attachments"
    params = {"f": "json", "token": TOKEN}
    r = requests.get(att_url, params=params)
    att_id = r.json()['attachmentInfos'][0]['id']

    # Download attachment
    file_url = f"{FEATURE_LAYER_URL}/{object_id}/attachments/{att_id}?token={TOKEN}"
    r = requests.get(file_url)
    filename = f"downloads/{object_id}_attachment.jpg"
    os.makedirs("downloads", exist_ok=True)
    with open(filename, 'wb') as f:
        f.write(r.content)
    return filename

In [108]:
# === STEP 3: SEND TO AI MODEL ===
def predict_pipe_material(image_path):
    with open(image_path, "rb") as img_file:
        res = requests.post(AI_MODEL_URL, files={"file": img_file})
        return res.json().get("prediction", "Unknown")

In [109]:
# === STEP 4: OCR + ADDRESS RAG VERIFICATION ===
def verify_address_with_database(image_path, reported_address):
    try:
        df = pd.read_excel(KNOWN_ADDRESSES_FILE)
        known_addresses = df["Address"].dropna().str.lower().tolist()

        # OCR to extract text
        ocr_text = pytesseract.image_to_string(Image.open(image_path)).lower()

        # Check if any known address is in OCR text
        for known in known_addresses:
            if known in ocr_text:
                return known  # ✅ Return matched correct address
        return None

    except Exception as e:
        print(f"OCR Verification Error: {e}")
        return None

In [110]:
# === STEP 5: ORCHESTRATION ===
def run_pipeline():
    survey_data, oid = get_latest_object_id()
    reported_address = survey_data.get("address", "").lower()

    img_path = download_latest_attachment(oid)
    prediction = predict_pipe_material(img_path)
    verified_address = verify_address_with_database(img_path, reported_address)

    print("\n📥 Survey Record:", survey_data)
    print("🔍 AI Prediction:", prediction)
    print("📍 Address Reported:", reported_address)
    print("✅ Verified Match:", verified_address if verified_address else "❌ No Match")

    if verified_address:
        print("🟢 All checks passed. Ready to upload to 120Water!")
    else:
        print("🔴 Address verification failed. Hold for manual review.")


if __name__ == "__main__":
    run_pipeline()


OCR Verification Error: 'Address'

📥 Survey Record: {'objectid': 2, 'globalid': '1fdabcd0-bf53-450d-8dd0-445a69792eb7', 'CreationDate': 1743613816696, 'Creator': 'thabo.traore_AAMU', 'EditDate': 1743613816696, 'Editor': 'thabo.traore_AAMU', 'staff_name': 'Ibrahim', 'field_survey_date': 1743613200000, 'service_type': 'Potholed', 'address': 'Huntsville, AL, USA', 'system_owned_material': 'Copper', 'system_owned_material_other': None, 'system_owned_notes': 'N/A', 'customer_owned_material': 'Copper', 'customer_owned_material_other': None, 'customer_owned_notes': 'N/A', 'lead_fittings': 'Yes', 'fittingssolder_notes': 'N/A'}
🔍 AI Prediction: PVC
📍 Address Reported: huntsville, al, usa
✅ Verified Match: ❌ No Match
🔴 Address verification failed. Hold for manual review.


In [111]:
import os

print("📁 Current Working Directory:", os.getcwd())


📁 Current Working Directory: c:\Users\PC\OneDrive\DOCUMENTS\GITHUB\arcgis\notebooks


In [112]:
import requests

# Base info
FEATURE_LAYER_URL = "https://services5.arcgis.com/s1DFLqWfkSIcXZ9O/ArcGIS/rest/services/survey123_24aa1572e50e42e8aa193dad894f57af_results/FeatureServer/0"
TOKEN = "mzFcMRqhxzPAoRJavp2MJg-w3Fau_D0Ix0Qviif4Gs2B6QMLbfdRhemy-rS5IGGgZCF2xD3K_WcpdZ8nn-17hZfwrssrW7Ne3VAAL7ZevueruBrhqVNxYL_dztEGkBKuCgs0zyzJxbkFs4rCovFSgb5aq2vxX-9Zkp3SFu3x3Sxo6q1cyxMxFkDL_jaC627O"

# Step 1: Get all OBJECTIDs
def get_object_ids():
    url = f"{FEATURE_LAYER_URL}/query"
    params = {
        "where": "1=1",
        "returnIdsOnly": "true",
        "f": "json",
        "token": TOKEN
    }
    r = requests.get(url, params=params)
    if r.status_code == 200:
        return r.json().get("objectIds", [])
    else:
        print("Error getting object IDs:", r.text)
        return []

# Step 2: Check for attachments
def get_attachments_for_object(object_id):
    url = f"{FEATURE_LAYER_URL}/{object_id}/attachments"
    params = {
        "f": "json",
        "token": TOKEN
    }
    r = requests.get(url, params=params)
    if r.status_code == 200:
        return r.json().get("attachmentInfos", [])
    return []

# Step 3: Loop and find those with images
def list_image_urls():
    object_ids = get_object_ids()
    all_image_links = []
    
    for oid in object_ids:
        attachments = get_attachments_for_object(oid)
        for att in attachments:
            image_url = f"{FEATURE_LAYER_URL}/{oid}/attachments/{att['id']}?token={TOKEN}"
            all_image_links.append({
                "object_id": oid,
                "filename": att["name"],
                "url": image_url
            })

    return all_image_links

# Main
if __name__ == "__main__":
    images = list_image_urls()
    if images:
        print("📸 Found image attachments:")
        for img in images:
            print(f"OID {img['object_id']}: {img['filename']} → {img['url']}")
    else:
        print("No image attachments found.")


No image attachments found.
